Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [0]:
import tensorflow as tf
import os
import zipfile

def train_emotion_model():
  
  DESIRED_ACCURACY = 0.999

  !wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

  zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
  zip_ref.extractall("/tmp/h-or-s")
  zip_ref.close()

  class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs = {}):
        if (logs.get('acc') > DESIRED_ACCURACY):
          print('\nReached 99.9% accuracy so cancelling training!')
          self.model.stop_training = True

  callbacks = myCallback()

In [0]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss = 'binary_crossentropy',
             optimizer = RMSprop(lr = 0.001),
             metrics = ['acc'])

In [0]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
    directory = '/tmp/h-or-s/',
    target_size = (150,150),
    batch_size = 128,
    class_mode = 'binary'
)
# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [0]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = 8,
    epochs = 15,
    callbacks = [callbacks]
)
      # Your Code Here)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 14s 2s/step - loss: 0.7107 - acc: 0.5688
Epoch 2/15
8/8 [==============================] - 12s 2s/step - loss: 0.4601 - acc: 0.8266
Epoch 3/15
8/8 [==============================] - 13s 2s/step - loss: 0.1940 - acc: 0.9250
Epoch 4/15
8/8 [==============================] - 12s 2s/step - loss: 0.0853 - acc: 0.9750
Epoch 5/15
8/8 [==============================] - 13s 2s/step - loss: 0.0473 - acc: 0.9828
Epoch 6/15
7/8 [=========================>....] - ETA: 1s - loss: 0.0121 - acc: 1.0000
Reached training accuracy over 99.9% so cancelling training!
8/8 [==============================] - 13s 2s/step - loss: 0.0121 - acc: 1.0000
